In [2]:
import numpy as np
import pandas as pd
import datetime

from cmf_engine.credentials import *
from cmf_engine.fetch_data import *
from cmf_engine.transform import *
from cmf_engine.infer import *

MODEL_PATH = os.path.abspath("./bin/model/")
# from cmf_engine.fetch_data import *

In [3]:
def main(counter,dff):
    print("---------  Counter: ", counter)
    path = 'C:/Users/bgondaliya/git/cmf-mvp3-quarter/bin/local/'
    
    # 0. get pond Credentials
    cred = credentials(path)
    cxn_ODS,cxn_6438 = cred.read_cred()
    
    # 1. to get the data
    data = TeleportData(cxn_6438,cxn_ODS)
    raw_df = data.get_data()
    oil_df = data.get_eia_data(cxn_ODS)
#     raw_df.to_csv('raw_df.csv')
#     oil_df.to_csv('oil_df.csv')
    
    # 2.define variables
    basins = ['AML', 'ASIA', 'MENA', 'OAT', 'RCA', 'Worldwide']
    
    
    
    counter = 0
    data_start_date = "2017-01-01"
    pickup_date = pd.to_datetime("2023-04-01",format="%Y-%m-%d")+ relativedelta(months=counter*3)#str(datetime.date(date.today().year, date.today().month, 1))
    quarter = (pickup_date.month - 1) // 3 + 1
    forecast_start_date = pickup_date.strftime('%Y Q{}'.format(quarter))
    refresh_date =  pd.to_datetime("2023-04-16",format="%Y-%m-%d")+ relativedelta(months=counter*3) #str(date.today())
    refresh_date = refresh_date.strftime('%Y Q{}'.format(quarter))
    duration_of_forecast = 12 #quarter



#     data_start_date = "2020-05-01"
#     forecast_start_date = str(pd.to_datetime("2022-03-01",format="%Y-%m-%d")+ relativedelta(months=counter))#str(datetime.date(date.today().year, date.today().month, 1))
#     refresh_date =  str(pd.to_datetime("2022-03-26",format="%Y-%m-%d")+ relativedelta(months=counter)) #str(date.today())
#     duration_of_forecast = 12
    
    
    raw_df = raw_df[raw_df["SL BASIN (CODE)"]!="SL_Elims"]
    raw_df["SL BASIN (CODE)"] = raw_df["SL BASIN (CODE)"]+"-"+raw_df["SL DIVISION (CODE)"]


    # Dep Vars
    rev_var = ["rev"]
    cm_var = ['cm']

    # Baseline Model

    rev_features = ['c4c_rev', 'gac_rev',
           'BC_OIL_PRICE_USD', 'WTI_OIL_PRICE_USD', 'gac_activity(t-1)',
           'gac_activity(t-2)', 'gac_activity(t-3)', 'gac_activity(t+1)',
           'gac_activity(t+2)',
           'c4c_activity(t-1)', 'c4c_activity(t-2)', 'c4c_activity(t-3)',
           'c4c_activity(t+1)', 'c4c_activity(t+2)', 'wti_activity(t-1)',
           'wti_activity(t-2)', 'wti_activity(t-3)', 'wti_activity(t+1)',
           'wti_activity(t+2)', 'btc_activity(t-1)', 'btc_activity(t-2)',
           'btc_activity(t-3)', 'btc_activity(t+1)', 'btc_activity(t+2)']



    cm_features = [
            'rev', 'c4c_rev', 'gac_rev',
           'BC_OIL_PRICE_USD', 'WTI_OIL_PRICE_USD', 'gac_activity(t-1)',
           'gac_activity(t-2)', 'gac_activity(t-3)', 'gac_activity(t+1)',
           'gac_activity(t+2)',
           'c4c_activity(t-1)', 'c4c_activity(t-2)', 'c4c_activity(t-3)',
           'c4c_activity(t+1)', 'c4c_activity(t+2)', 'wti_activity(t-1)',
           'wti_activity(t-2)', 'wti_activity(t-3)', 'wti_activity(t+1)',
           'wti_activity(t+2)', 'btc_activity(t-1)', 'btc_activity(t-2)',
           'btc_activity(t-3)', 'btc_activity(t+1)', 'btc_activity(t+2)',
           'rev_activity(t-1)', 'rev_activity(t-2)', 'rev_activity(t-3)']

    # Prophet Model
    pr_rev_features = ['c4c_rev', 'gac_rev',
           'BC_OIL_PRICE_USD', 'WTI_OIL_PRICE_USD',
            'gac_activity(t-1)',
           'gac_activity(t-2)', 'gac_activity(t-3)', 'gac_activity(t+1)',
           'gac_activity(t+2)',
           'c4c_activity(t-1)', 'c4c_activity(t-2)', 'c4c_activity(t-3)',
           'c4c_activity(t+1)', 'c4c_activity(t+2)', 'wti_activity(t-1)',
           'wti_activity(t-2)', 'wti_activity(t-3)', 'wti_activity(t+1)',
           'wti_activity(t+2)', 'btc_activity(t-1)', 'btc_activity(t-2)',
           'btc_activity(t-3)', 'btc_activity(t+1)', 'btc_activity(t+2)',
                    ]
    
    pr_cm_features = ['c4c_rev', 'gac_rev','rev',
           'BC_OIL_PRICE_USD', 'WTI_OIL_PRICE_USD', 
            'gac_activity(t-1)',
           'gac_activity(t-2)', 'gac_activity(t-3)', 'gac_activity(t+1)',
           'gac_activity(t+2)', 
           'c4c_activity(t-1)', 'c4c_activity(t-2)', 'c4c_activity(t-3)',
           'c4c_activity(t+1)', 'c4c_activity(t+2)', 'wti_activity(t-1)',
           'wti_activity(t-2)', 'wti_activity(t-3)', 'wti_activity(t+1)',
           'wti_activity(t+2)', 'btc_activity(t-1)', 'btc_activity(t-2)',
           'btc_activity(t-3)', 'btc_activity(t+1)', 'btc_activity(t+2)',
           'rev_activity(t-1)', 'rev_activity(t-2)', 'rev_activity(t-3)',]
    

    # Neural Prophet Model

    npr_features = ['c4c_rev', 'gac_rev',
           'BC_OIL_PRICE_USD', 'WTI_OIL_PRICE_USD', 
            'gac_activity(t-1)',
           'gac_activity(t-2)', 'gac_activity(t-3)', 'gac_activity(t+1)',
           'gac_activity(t+2)', 
           'c4c_activity(t-1)', 'c4c_activity(t-2)', 'c4c_activity(t-3)',
           'c4c_activity(t+1)', 'c4c_activity(t+2)', 'wti_activity(t-1)',
           'wti_activity(t-2)', 'wti_activity(t-3)', 'wti_activity(t+1)',
           'wti_activity(t+2)', 'btc_activity(t-1)', 'btc_activity(t-2)',
           'btc_activity(t-3)', 'btc_activity(t+1)', 'btc_activity(t+2)',]


    lagged_features = ['rev','rev_activity(t-1)', 'rev_activity(t-2)', 'rev_activity(t-3)']


    basins = ['AML-DAI',
     'AML-MSP',
     'AML-N/A',
     'AML-NEO',
     'AML-OTH',
     'AML-PRS',
     'AML-RPF',
     'AML-WEC',
     'ASIA-DAI',
     'ASIA-MSP',
     'ASIA-N/A',
     'ASIA-NEO',
     'ASIA-OTH',
     'ASIA-PRS',
     'ASIA-RPF',
     'ASIA-WEC',
     'GHQ-DAI',
     'GHQ-MSP',
     'GHQ-NEO',
     'GHQ-OTH',
     'GHQ-PRS',
     'GHQ-RPF',
     'GHQ-WEC',
     'MENA-DAI',
     'MENA-MSP',
     'MENA-N/A',
     'MENA-NEO',
     'MENA-OTH',
     'MENA-PRS',
     'MENA-RPF',
     'MENA-WEC',
     'N/A-DAI',
     'OAT-DAI',
     'OAT-MSP',
     'OAT-N/A',
     'OAT-NEO',
     'OAT-OTH',
     'OAT-PRS',
     'OAT-RPF',
     'OAT-WEC',
     'RCA-DAI',
     'RCA-MSP',
     'RCA-OTH',
     'RCA-PRS',
     'RCA-RPF',
     'RCA-WEC',
     'Worldwide-DAI',
     'Worldwide-MSP',
     'Worldwide-N/A',
     'Worldwide-NEO',
     'Worldwide-OTH',
     'Worldwide-PRS',
     'Worldwide-RPF',
     'Worldwide-WEC'
        ]
    
    
    # 3. transform the data for Rev
    ## RF
    tt = TrainingTransformer(data_start_date,duration_of_forecast,basins)
    
    
    tt.fit(raw_df)
    
    train_df = tt.transform(raw_df,oil_df)
    
    train_df = train_df[train_df.index<forecast_start_date]
    pr_train_df = train_df.copy()
    pr_train_df = pr_train_df.reset_index()

    it = InferenceRevTransformer(12,pickup_date,basins)
    it.fit(raw_df)
    infer_df = it.transform(raw_df,oil_df)
    infer_df.to_csv("infer_df.csv")
    pr_infer_df = infer_df.copy()
    pr_infer_df = pr_infer_df.reset_index()

    npr_train_df = pr_train_df.copy()
    npr_infer_df = pr_infer_df.copy()
    npr_infer_df.to_csv("npr_infer_df.csv")

    train_basins = set([
         'MENA-WEC',
         'ASIA-DAI',
         'OAT-RPF',
         'AML-WEC',
         'AML-DAI',
         'RCA-WEC',
         'OAT-WEC',
         'ASIA-RPF',
         'ASIA-PRS',
         'ASIA-WEC',
         'RCA-RPF',
         'RCA-DAI',
         'AML-PRS',
         'AML-RPF',
         'OAT-PRS',
         'MENA-PRS',
         'OAT-DAI',
         'MENA-RPF',
         'RCA-PRS',
         'MENA-DAI',
        'Worldwide-DAI','Worldwide-WEC','Worldwide-PRS','Worldwide-RPF'
    ])

    # Part-3
    ### RF Model
    rev_model_path = os.path.join(MODEL_PATH, "rev", "RF")
    print('path----',rev_model_path)
    rev_trainer = RfModel(rev_features,rev_var,train_basins,pickup_date,rev_model_path)
    rev_trainer.train(train_df)
    rev_data_dict = rev_trainer.infer(infer_df)


    pr_rev_model_path = os.path.join(MODEL_PATH, "rev", "Prophet")
    pr_rev_trainer = ProphetModel(pr_rev_features,pr_rev_features,rev_var,train_basins,pickup_date,pr_rev_model_path)
    pr_rev_trainer.train(pr_train_df)
    pr_rev_data_dict = pr_rev_trainer.infer(pr_infer_df)


    npr_rev_model_path = os.path.join(MODEL_PATH, "rev", "NeuralPr")
    npr_rev_trainer = NeuralProphetModel(npr_features,npr_features,rev_var[0],[],
                                         train_basins,pickup_date,rev_model_path)
    npr_train = npr_rev_trainer.train(npr_train_df)
    npr_rev_data_dict = npr_rev_trainer.infer(npr_infer_df, npr_train)


if __name__ == "__main__":
#     counter = 0
#     main(counter)
    for counter in range(1,):
        dff = pd.DataFrame(columns = ['level', 'date', 'YEARMONTH', 'actual_revenue', 'predicted_revenue',
       'revenue_error%', 'actual_cm', 'predicted_cm', 'cm_error%',
       'refresh_date'])
        main(counter,dff)
        

---------  Counter:  0


WARNING - (py.warnings._showwarnmsg) - C:\Users\bgondaliya\git\cmf-mvp3-quarter\cmf_engine\fetch_data.py:66: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  c4c_df = pd.read_sql(c4c_query,self.cnx)

WARNING - (py.warnings._showwarnmsg) - C:\Users\bgondaliya\git\cmf-mvp3-quarter\cmf_engine\fetch_data.py:67: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  gac_df = pd.read_sql(gac_query,self.cnx)

WARNING - (py.warnings._showwarnmsg) - C:\Users\bgondaliya\git\cmf-mvp3-quarter\cmf_engine\fetch_data.py:68: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider 

path---- C:\Users\bgondaliya\git\cmf-mvp3-quarter\bin\model\rev\RF


  0%|                                                                                            | 0/2 [00:00<?, ?it/s]WARNING - (py.warnings._showwarnmsg) - C:\Users\bgondaliya\git\cmf-mvp3-quarter\cmf_engine\infer.py:75: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  regressor.fit(X=X_train,y=yt_train)#,sample_weight = weights)



days_before_recent_date: Int64Index([1461, 1461, 1186, 1186, 1155, 1155, 1127, 1127, 1096, 1096,  821,
             821,  790,  790,  761,  761,  730,  730,  455,  455,  424,  424,
             396,  396,  365,  365,   90,   90,   59,   59,   31,   31,    0,
               0],
           dtype='int64')


...

INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 3.09E-02, min: 2.45E-01


  0%|          | 0/101 [00:00<?, ?it/s]

INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 8.32E+00, min: 1.29E+00
INFO - (NP.forecaster._init_train_loader) - lr-range-test selected learning rate: 2.95E-01
Epoch[1000/1000]: 100%|█| 1000/1000 [00:14<00:00, 68.00it/s, SmoothL1Loss=9.89e-8, MAE=58.8, RMSE=98.5, Loss=9.68e-8, R
INFO - (NP.df_utils._infer_frequency) - Major frequency QS corresponds to 76.471% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - QS
WARNING - (NP.forecaster._make_future_dataframe) - Number of forecast steps is defined by n_forecasts. Adjusted to 4.
INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
INFO - (NP.df_utils._infer_frequency) - Major frequency QS corresponds to 62.5% of the data.
WARNING - (NP.df_utils._infer_frequency) - Dataframe has multiple frequencies. It will be resampled according to given freq QS. Ignore message if actual frequency is any of the following:  SM, BM, CBM, SMS, BMS, CBM

ASIA-DAI --------------> Regressors
['c4c_rev', 'gac_rev', 'BC_OIL_PRICE_USD', 'WTI_OIL_PRICE_USD', 'gac_activity(t-1)', 'gac_activity(t-2)', 'gac_activity(t-3)', 'gac_activity(t+1)', 'gac_activity(t+2)', 'c4c_activity(t-1)', 'c4c_activity(t-2)', 'c4c_activity(t-3)', 'c4c_activity(t+1)', 'c4c_activity(t+2)', 'wti_activity(t-1)', 'wti_activity(t-2)', 'wti_activity(t-3)', 'wti_activity(t+1)', 'wti_activity(t+2)', 'btc_activity(t-1)', 'btc_activity(t-2)', 'btc_activity(t-3)', 'btc_activity(t+1)', 'btc_activity(t+2)'] Future Regressors
MENA-WEC --------------> Regressors
['c4c_rev', 'gac_rev', 'BC_OIL_PRICE_USD', 'WTI_OIL_PRICE_USD', 'gac_activity(t-1)', 'gac_activity(t-2)', 'gac_activity(t-3)', 'gac_activity(t+1)', 'gac_activity(t+2)', 'c4c_activity(t-1)', 'c4c_activity(t-2)', 'c4c_activity(t-3)', 'c4c_activity(t+1)', 'c4c_activity(t+2)', 'wti_activity(t-1)', 'wti_activity(t-2)', 'wti_activity(t-3)', 'wti_activity(t+1)', 'wti_activity(t+2)', 'btc_activity(t-1)', 'btc_activity(t-2)', 'btc_

WARNING - (NP.forecaster._make_future_dataframe) - Number of forecast steps is defined by n_forecasts. Adjusted to 4.
INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
INFO - (NP.df_utils._infer_frequency) - Major frequency QS corresponds to 62.5% of the data.
WARNING - (NP.df_utils._infer_frequency) - Dataframe has multiple frequencies. It will be resampled according to given freq QS. Ignore message if actual frequency is any of the following:  SM, BM, CBM, SMS, BMS, CBMS, BQ, BQS, BA, or, BAS.
INFO - (NP.df_utils._infer_frequency) - Major frequency QS corresponds to 62.5% of the data.
WARNING - (NP.df_utils._infer_frequency) - Dataframe has multiple frequencies. It will be resampled according to given freq QS. Ignore message if actual frequency is any of the following:  SM, BM, CBM, SMS, BMS, CBMS, BQ, BQS, BA, or, BAS.
INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column


In [4]:
a

NameError: name 'a' is not defined